### **Zaawansowane bazy danych i hurtownie danych**

## **Praca domowa #5: MongoDB**

#### **Autor:** _Piotr Zajączkowski_


In [2]:
import pandas as pd
import pymongo as pm
from pymongo import MongoClient

# podłączenie do mongodb
client = MongoClient("localhost", 27017)
db = client.IMDB

title = db["Title"]
rating = db["Rating"]
name = db["Name"]

### Zadanie 1

Sprawdź liczbę dokumentów w kolekcjach Title/Rating/Name.


In [3]:
z1_title_count = title.count_documents({})
z1_rating_count = rating.count_documents({})
z1_name_count = name.count_documents({})

print(f"Liczba dokumentów w kolekcji 'Title': {z1_title_count}")
print(f"Liczba dokumentów w kolekcji 'Rating': {z1_rating_count}")
print(f"Liczba dokumentów w kolekcji 'Name': {z1_name_count}")

Liczba dokumentów w kolekcji 'Title': 10803675
Liczba dokumentów w kolekcji 'Rating': 1440952
Liczba dokumentów w kolekcji 'Name': 13525057


### Zadanie 2

Wybierz 4 pierwsze dokumenty z kolekcji Title, które były wyprodukowane w roku 2020, są z kategorii filmów
Romance, ich czas trwania jest większy niż 90 minut, ale nie przekracza 120 minut. Zwracane dokumenty powinny
zawierać tytuł, rok produkcji, kategorię oraz czas trwania. Dane uporządkuj rosnąco wg tytułu filmu. Sprawdź
również, ile dokumentów zwróciłoby zapytanie po wyłączeniu ograniczenia w postaci 4 pierwszych dokumentów.
Wyszukując łańcuchy skorzystaj z operatora $regex.


In [4]:
z2_results = (
    title.find(
        {
            "startYear": 2020,
            "genres": {"$regex": "Romance"},
            "runtimeMinutes": {"$gt": 90, "$lte": 120},
        },
        {"_id": 0, "primaryTitle": 1, "startYear": 1, "genres": 1, "runtimeMinutes": 1},
    )
    .sort("primaryTitle", pm.ASCENDING)
    .limit(4)
)

print("Pierwsze 4 dokumenty spełniające kryteria:")
pd.DataFrame(z2_results)

Pierwsze 4 dokumenty spełniające kryteria:


,primaryTitle,startYear,runtimeMinutes,genres
0,419,2020,111,"Drama,Romance,Thriller"
1,#s_uchilishcha,2020,95,"Drama,Romance"
2,2 Hearts,2020,101,"Drama,Romance"
3,23 Walks,2020,102,"Comedy,Drama,Romance"


In [5]:
z2_total_count = title.count_documents(
    {
        "startYear": 2020,
        "genres": {"$regex": "Romance"},
        "runtimeMinutes": {"$gt": 90, "$lte": 120},
    }
)

print(f"Liczba dokumentów bez ograniczenia: {z2_total_count}")

Liczba dokumentów bez ograniczenia: 490


### Zadanie 3

Sprawdź ile filmów różnego typu (pole titleType) było wprodukowanych w roku 2000. Wynik zapytania
powinien zwracać nazwę typu oraz liczbę filmów.


In [6]:
z3_result = title.aggregate(
    [
        {"$match": {"startYear": 2000}},
        {"$group": {"_id": "$titleType", "ilość": {"$sum": 1}}},
        {"$project": {"typFilmu": "$_id", "ilość": 1, "_id": 0}},
    ]
)
pd.DataFrame(z3_result)

,ilość,typFilmu
0,5245,movie
1,656,tvSpecial
2,765,videoGame
3,121,tvShort
4,4867,short
5,5279,video
6,2469,tvMovie
7,78162,tvEpisode
8,2983,tvSeries
9,372,tvMiniSeries


# Zadanie 4

W oparciu o kolekcje Title oraz Rating sprawdź średnią ocenę filmów dokumentalnych wprodukowanych w
latach 2010-2012. Wyświetl tytuł filmu, rok produkcji oraz jego średnią ocenę. Dane uporzadkuj malejąco wg
średniej oceny.

- sprawdź, ile takich dokumentów zwróci zapytanie

- wyświetl tylko 5 pierwszych dokumentów spełniających powyższe warunki


In [7]:
# zadanie robiło się bardzo długo, więc musiałem dodać indeksy
rating.create_index("tconst", unique=True)
title.create_index("tconst", unique=True)

z4_results = title.aggregate(
    [
        {
            "$match": {
                "startYear": {"$gte": 2010, "$lte": 2012},
                "genres": {"$regex": "Documentary"},
            }
        },
        {
            "$lookup": {
                "from": "Rating",
                "localField": "tconst",
                "foreignField": "tconst",
                "as": "rating",
            }
        },
        {"$unwind": "$rating"},
        {"$sort": {"rating.averageRating": pm.DESCENDING}},
        {"$limit": 5},
        {
            "$project": {
                "_id": 0,
                "primaryTitle": 1,
                "startYear": 1,
                "genres": 1,
                "rating": "$rating.averageRating",
            }
        },
    ]
)

pd.DataFrame(z4_results)

,primaryTitle,startYear,genres,rating
0,Tripping on Hallucinogenic Frogs,2012,Documentary,10.0
1,"Mitropanos, Kotsiras, Basis Live 06-07-2011",2012,"Biography,Comedy,Documentary",10.0
2,Episode #9.26,2010,Documentary,10.0
3,Episode #8.62,2010,Documentary,10.0
4,Nikki & Daniel,2012,"Documentary,Reality-TV,Romance",10.0


# Zadanie 5

Utwórz indeks tekstowy dla pola primaryName w kolekcji Name. Następnie używając tego indeksu znajdź
dokumenty opisujące osoby o nazwisku Fonda oraz Coppola. Przy wyszukiwaniu włącz opcję, która będzie
uwzględniać wielkie/małe litery.

- Ile dokumentów zwraca zapytanie?

- Wyświetl 5 pierwszych dokumentów (dokument powinien zwracać dwa pola: primaryName,
  primaryProfession).


In [14]:
name.create_index([("primaryName", "text")])

z5_query = {"$text": {"$search": "Fonda Coppola", "$caseSensitive": False}}
z5_result = name.find(
    z5_query,
    {"_id": 0, "primaryName": 1, "primaryProfession": 1},
).limit(5)

z5_result_count = name.count_documents(z5_query)

print(f"Ilość zwróconych dokumentów: {z5_result_count}")

pd.DataFrame(z5_result)

Ilość zwróconych dokumentów: 667


,primaryName,primaryProfession
0,Sabrina Coppola,"script_department,actress,writer"
1,Andrew Coppola,\N
2,Vicki Coppola,actress
3,Neil Coppola,\N
4,Cassandra Coppola,"production_manager,assistant_director,producer"


# Zadanie 6

Utwórz indeks z porządkiem malejącym dla pola birthYear (kolekcja Name). Następnie wyświetl listę indeksów
w kolekcji Name. Ile indeksów posiada ta kolekcja?


In [25]:
name.create_index([("birthYear", pm.DESCENDING)])

z6_indexes = name.list_indexes()

z6_indexes_frame = pd.DataFrame(z6_indexes)
print(f"Ilość indeksów w kolekcji name: {z6_indexes_frame.shape[0]}")
z6_indexes_frame

Ilość indeksów w kolekcji name: 3


,v,key,name,weights,default_language,language_override,textIndexVersion
0,2,{'_id': 1},_id_,NaN,NaN,NaN,NaN
1,2,"{'_fts': 'text', '_ftsx': 1}",primaryName_text,{'primaryName': 1},english,language,3.0
2,2,{'birthYear': -1},birthYear_-1,NaN,NaN,NaN,NaN


# Zadanie 7

Dla każdego filmu (kolekcja Title), który ma najwyższą średnią ocenę (10.0), dodaj pole max z wartością równą 1. W poleceniu skorzystaj z kolekcji Rating, która zawiera informacje o średniej ocenie filmu.


In [29]:
title.update_many(
    {"tconst": {"$in": rating.distinct("tconst", {"averageRating": 10})}},
    {"$set": {"max": 1}},
)

z7_result = title.find({"max": 1}, {"_id": 0, "tconst": 1, "primaryTitle": 1, "max": 1})

pd.DataFrame(z7_result)

,tconst,primaryTitle,max
0,tt0058271,Koll,1
1,tt0058559,Sechs Personen suchen einen Autor,1
2,tt0065800,Eine große Familie,1
3,tt0127236,Renegades 2,1
4,tt0143979,A View of Bosnia,1
...,...,...,...
6717,tt9805208,I Do Not Know What City Happens in His Eyes,1
6718,tt9816188,The Curfew,1
6719,tt9866444,Ask Aldo,1
6720,tt9889628,Fiat & Alfa Romeo Racers,1


# Zadanie 8

W oparciu o kolekcje Title oraz Rating sprawdź średnią ocenę dowolnego filmu o określonym tytule oraz roku
produkcji. Zapytanie powinno zwrócić nazwę filmu, rok produkcji oraz średnią ocenę.


In [31]:
z8_movie_title = "Lifepod"
z8_rok_produkcji = 2020

result = title.aggregate(
    [
        {"$match": {"primaryTitle": z8_movie_title, "startYear": z8_rok_produkcji}},
        {
            "$lookup": {
                "from": "Rating",
                "localField": "tconst",
                "foreignField": "tconst",
                "as": "rating",
            }
        },
        {"$unwind": "$rating"},
        {
            "$project": {
                "_id": 0,
                "Nazwa filmu": "$primaryTitle",
                "Rok produkcji": "$startYear",
                "Średnia ocena": "$rating.averageRating",
            }
        },
    ]
)

pd.DataFrame(result)

,Nazwa filmu,Rok produkcji,Średnia ocena
0,Lifepod,2020,4.7


# Zadanie 9

Do filmu Blade Runner z roku 1982 dodaj pole rating, które będzie tablicą dokumentów z polami:
averageRating oraz numVotes. Wartości pól w zagnieżdżonym dokumencie powinny posiadać odpowiednie
wartości pobrane z kolekcji Rating.


In [34]:
title.aggregate(
    [
        {"$match": {"primaryTitle": "Blade Runner", "startYear": 1982}},
        {
            "$lookup": {
                "from": "Rating",
                "localField": "tconst",
                "foreignField": "tconst",
                "as": "joinedRatings",
            },
        },
        {"$unwind": "$joinedRatings"},
        {
            "$project": {
                "rating": ["$joinedRatings.averageRating", "$joinedRatings.numVotes"],
            }
        },
        {"$merge": {"into": "title", "on": "_id", "whenMatched": "merge"}},
    ]
)

z9_result = title.find({"primaryTitle": "Blade Runner", "startYear": 1982})
pd.DataFrame(z9_result)

,_id,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,66561f730834a89fb470e2af,tt0083658,movie,Blade Runner,Blade Runner,0,1982,\N,117,"Action,Drama,Sci-Fi"


# Zadanie 10


# Zadanie 11


# Zadanie 12


# Zadanie 13
